## Fixed Penalty Matrix Factorization (FP)

Paper : Online forecasting matrix factorization - Gultekin & Paisley
Nous avons implémenté l'algorithme 1(p4) . 

Nous avons pris une partie de données sur la consommation electrique par 15min utilisée dans la partie exprimentale du papier. Vous trouverez jointe la base de données tronquée utilisée. 

**Question : A quoi correspond la matrice U_barre(:,I_t) ? Nous ne comprenons pas malgré la note 1.** Nous n'avons donc pas pu initialiser le U_barre comme à la ligne 3 de l'algorithme et updater U_t (ligne 8)


In [171]:
import numpy as np
import pandas as pd

In [172]:
# Import data: database electricité tronquée car trop long
raw_data=pd.read_csv("database_electricite_tronquee.csv",sep=';')
data=raw_data.copy()
data.head()

,31/12/2011 23:45,01/01/2012 00:00,01/01/2012 00:15,01/01/2012 00:30,01/01/2012 00:45,01/01/2012 01:00,01/01/2012 01:15,01/01/2012 01:30,01/01/2012 01:45,01/01/2012 02:00,...,01/01/2012 15:00,01/01/2012 15:15,01/01/2012 15:30,01/01/2012 15:45,01/01/2012 16:00,01/01/2012 16:15,01/01/2012 16:30,01/01/2012 16:45,01/01/2012 17:00,01/01/2012 17:15
0,0,0,3.807107,5.076142,3.807107,3.807107,5.076142,3.807107,3.807107,6.345178,...,15.228426,16.497462,15.228426,11.421320,3.807107,2.538071,1.269036,2.538071,2.538071,3.807107
1,0,0,22.759602,22.759602,22.759602,22.759602,22.048364,22.759602,22.759602,24.893314,...,28.449502,27.738265,27.738265,26.315789,27.027027,26.315789,26.315789,26.315789,25.604552,26.315789
2,0,0,77.324066,77.324066,77.324066,77.324066,77.324066,77.324066,77.324066,77.324066,...,3.475239,3.475239,3.475239,3.475239,2.606429,3.475239,3.475239,3.475239,3.475239,2.606429
3,0,0,136.178862,136.178862,140.243902,140.243902,146.341463,134.146342,132.113821,136.178862,...,73.170732,75.203252,73.170732,73.170732,89.430894,87.398374,89.430894,89.430894,87.398374,87.398374
4,0,0,70.731707,73.170732,69.512195,75.609756,73.170732,73.170732,67.073171,67.073171,...,41.463415,41.463415,45.121951,43.902439,48.780488,45.121951,48.780488,50.000000,47.560976,45.121951


In [173]:
# Initialisation comme dans le papier de recherche 
d = 5 # à optimiser par crossvalidation plus tard
rho_v =pow(10,-4)
rho_u =1
max_ite=15


M=data.shape[0]
T=data.shape[1]
columns=data.columns

U_0=np.random.rand(d,M)
v_0=np.random.rand(d,1)

# on teste l'algorithme au temps t=4 pour voir s'il tourne
t=4
x_t=np.asmatrix(data[columns[t-1]])


In [165]:
# Fonction de l'algorithme : Fixed penalty Matrix Factorization
def FP(x_t,I_t,rho_u,rho_v, U_barre, v_barre , max_ite):
    
    U_b = U_barre # à corriger
    U = U_barre
    for i in range(1,max_ite+1):
        v=(np.linalg.inv(rho_v*np.eye(d)+U.dot(np.transpose(U))))*(rho_v*v_barre+U.dot(np.transpose(x_t)))
        U=(np.linalg.inv(rho_u*np.eye(d)+v.dot(np.transpose(v))))*(rho_u*U_barre+v.dot(x_t))
        
    U_t=U # à corriger
    v_t=v

    return U_t,v_t

In [166]:
nv_U, nv_v = FP(x_t,1,rho_u,rho_v, U_0, v_0 , max_ite)